In [1]:
import pandas as pd
import numpy as np

In [ ]:
x_train = pd.read_csv('WaterPumpData/features_train.csv')
y_train = pd.read_csv('WaterPumpData/labels_train.csv')
y_train['label'] = y_train.status_group.apply(lambda x: 1 if x == 'functional' else 0)

In [352]:
import random
from random import sample

validation_set = sample(list(x_train.id), len(x_train)//5)

In [353]:
from sklearn import preprocessing
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def create_gradientbooster_model(feature_set, cat_var):
    categorical_bool = [True if i in cat_var else False for i in feature_set]
    le = preprocessing.LabelEncoder()
    xx_train = x_train.copy()
    xx_train[cat_var] = xx_train[cat_var].apply(le.fit_transform)
    xxx_train = xx_train[feature_set].loc[~xx_train.id.isin(validation_set)]
    xxx_validate = xx_train[feature_set].loc[xx_train.id.isin(validation_set)]
    yy_train = y_train.loc[~y_train.id.isin(validation_set)].label
    yy_validate = y_train.loc[y_train.id.isin(validation_set)].label
    clf = HistGradientBoostingClassifier(categorical_features=categorical_bool)
    clf.fit(xxx_train, yy_train)
    
    return clf, xxx_validate, pd.DataFrame(yy_validate)

In [122]:
feature_set_1 = ['gps_height', 'basin', 'region', 'population', 'permit', 'construction_year', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
categorical_variables = ['basin', 'region', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group', 'permit']

model, xxx_validate, yy_validate = create_gradientbooster_model(feature_set_1, categorical_variables)

In [123]:
pred = model.predict(xxx_validate)
yy_validate['pred'] = pred
(yy_validate.pred == yy_validate.label).value_counts()

True     9417
False    2463
dtype: int64

In [135]:
feature_set_2 = ['gps_height', 'basin', 'region_code', 'public_meeting', 'population', 'permit', 'scheme_management', 'construction_year', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source', 'waterpoint_type']
categorical_variables = ['basin', 'region_code', 'public_meeting', 'extraction_type_group', 'scheme_management', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source', 'waterpoint_type', 'permit']

model, xxx_validate, yy_validate = create_gradientbooster_model(feature_set_2, categorical_variables)

In [136]:
pred = model.predict(xxx_validate)
yy_validate['pred'] = pred
(yy_validate.pred == yy_validate.label).value_counts()

True     9427
False    2453
dtype: int64

In [159]:
funder = x_train.funder.str.lower().value_counts()
funder = funder[funder > 250]
x_train['funder_mod'] = [i if i in funder else 'Unknown' for i in x_train.funder]

In [171]:
feature_set_3 = ['amount_tsh', 'population', 'funder_mod', 'district_code', 'basin', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']
categorical_variables = ['funder_mod', 'district_code', 'basin', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']

model, xxx_validate, yy_validate = create_gradientbooster_model(feature_set_3, categorical_variables)
pred = model.predict(xxx_validate)
yy_validate['pred'] = pred
(yy_validate.pred == yy_validate.label).value_counts()

True     9493
False    2387
dtype: int64